# Model Learning on Dataset from a Simulated Experiment

In this notebook, we will use a dataset from a simulated experiment, more specifically, the `Simulated_calibration.ipynb` example notebook and perform Model Learning on a simple 1 qubit model.

### Imports

In [1]:
import pickle
from pprint import pprint

## The Dataset

We first take a look below at the dataset and its properties. To explore more details about how the dataset is generated, please refer to the `Simulated_calibration.ipynb` example notebook.

In [2]:
DATAFILE_PATH = "data/small_dataset.pkl"

In [3]:
with open(DATAFILE_PATH, "rb+") as file:
    data = pickle.load(file)

In [4]:
data.keys()

dict_keys(['seqs_grouped_by_param_set', 'opt_map'])

Since this dataset was obtained from an ORBIT ([arXiv:1403.0035](https://arxiv.org/abs/1403.0035)) calibration experiment, we have the `opt_map` which will tell us about the gateset parameters being optimized.

In [5]:
data['opt_map']

[['rx90p[0]-d1-gauss-amp',
  'ry90p[0]-d1-gauss-amp',
  'rx90m[0]-d1-gauss-amp',
  'ry90m[0]-d1-gauss-amp'],
 ['rx90p[0]-d1-gauss-delta',
  'ry90p[0]-d1-gauss-delta',
  'rx90m[0]-d1-gauss-delta',
  'ry90m[0]-d1-gauss-delta'],
 ['rx90p[0]-d1-gauss-freq_offset',
  'ry90p[0]-d1-gauss-freq_offset',
  'rx90m[0]-d1-gauss-freq_offset',
  'ry90m[0]-d1-gauss-freq_offset'],
 ['id[0]-d1-carrier-framechange']]

This `opt_map` implies the calibration experiment focussed on optimizing 
the amplitude, delta and frequency offset of the gaussian pulse, along 
with the framechange angle

Now onto the actual measurement data from the experiment runs

In [6]:
seqs_data = data['seqs_grouped_by_param_set']

**How many experiment runs do we have?**

In [7]:
len(seqs_data)

41

**What does the data from each experiment look like?**

We take a look at the first data point

In [8]:
example_data_point = seqs_data[0]

In [9]:
example_data_point.keys()

dict_keys(['params', 'seqs', 'results', 'results_std', 'shots'])

These `keys` are useful in understanding the structure of the dataset. We look at them one by one.

In [10]:
example_data_point['params']

[450.000 mV, -1.000 , -50.500 MHz 2pi, 4.084 rad]

These are the parameters for our parameterised gateset, for the first experiment run. They correspond to the optimization parameters we previously discussed. 

The `seqs` key stores the sequence of gates that make up this ORBIT calibration experiment. Each ORBIT sequence consists of a set of gates, followed by a measurement operation. This is then repeated for some `n` number of shots (eg, `1000` in this case) and we only store the averaged result along with the standard deviation of these readout shots. Each experiment in turn consists of a number of these ORBIT sequences. The terms *sequence*, *set* and *experiment* are used somewhat loosely here, so we show below what these look like.

**A single ORBIT sequence**

In [11]:
example_data_point['seqs'][0]

['ry90p[0]',
 'rx90p[0]',
 'rx90p[0]',
 'rx90m[0]',
 'ry90p[0]',
 'ry90p[0]',
 'rx90p[0]',
 'ry90p[0]',
 'rx90p[0]',
 'rx90p[0]',
 'ry90p[0]',
 'rx90m[0]',
 'rx90p[0]',
 'rx90p[0]',
 'ry90p[0]',
 'ry90p[0]',
 'rx90p[0]',
 'ry90p[0]',
 'ry90m[0]',
 'rx90p[0]',
 'rx90p[0]',
 'ry90m[0]',
 'rx90p[0]',
 'rx90p[0]',
 'rx90p[0]',
 'rx90p[0]']

**Total number of ORBIT sequences in an experiment**

In [12]:
len(example_data_point['seqs'])

20

**Total number of Measurement results**

In [13]:
len(example_data_point['results'])

20

**The measurement results and the standard deviation look like this**

In [14]:
example_results = [(example_data_point['results'][i], 
                    example_data_point['results_std'][i]) 
                   for i in range(len(example_data_point['results']))]

In [15]:
pprint(example_results)

[([0.745], [0.013783141876945182]),
 ([0.213], [0.012947239087929134]),
 ([0.137], [0.0108734079294396]),
 ([0.224], [0.013184233007649706]),
 ([0.434], [0.015673034167001616]),
 ([0.105], [0.009694070352540258]),
 ([0.214], [0.012969348480166613]),
 ([0.112], [0.009972762907038352]),
 ([0.318], [0.014726710426975877]),
 ([0.122], [0.010349685985574633]),
 ([0.348], [0.015063067416698366]),
 ([0.122], [0.010349685985574633]),
 ([0.558], [0.01570464899321217]),
 ([0.186], [0.01230463327369004]),
 ([0.096], [0.009315793041926168]),
 ([0.368], [0.015250442616527561]),
 ([0.146], [0.011166198995181842]),
 ([0.121], [0.010313049985334118]),
 ([0.748], [0.013729384545565035]),
 ([0.692], [0.01459917805905524])]


## The Model for Model Learning

An initial model needs to be provided, which we refine by fitting to our calibration data. We do this below. If you want to learn more about what the various components of the model mean, please refer back to the `two_qubits.ipynb` notebook or the documentation.

### Define Constants

### Model

### Generator

### Gateset

### Experiment

### Optimizer 

## Model Learning

We are now ready to learn from the data and improve our model

### Result of Model Learning

## Visualisation & Analysis of Results

The Model Learning logs provide a useful way to visualise the learning process and also understand what's going wrong (or right). We now process these logs to read some data points and also plot some visualisations of the Model Learning process

### Open, Clean-up and Convert Logfiles

### Summary of Logs

### Qubit Anharmonicity

### Qubit Frequency

### Goal Function